In [78]:
import torch
from transformers import BertTokenizer, BertModel
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
import tqdm
import os
import time
import re
import pandas as pd
import string
import time
import collections
from collections import Counter
from nltk.corpus import stopwords
from itertools import chain
from sklearn.metrics import accuracy_score
import time 
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [79]:
tokenizer=BertTokenizer.from_pretrained(r'D:\conda\envs\xie\chinese-bert-wwm-ext')
bert = BertModel.from_pretrained(r'D:\conda\envs\xie\chinese-bert-wwm-ext')

In [181]:
data=pd.read_csv("yqdata.csv",usecols=['title','fact','opinion','inhibition','fff',
                                      'activation','moral'])

In [182]:
data['title']=data['title'].apply(lambda x:x[:15])

In [183]:
data['preprocess']=['[CLS] ' + sent + ' [SEP]' for sent in data['title'].values]
data['tokenized']=[tokenizer.tokenize(sent) for sent in data['preprocess']]
data['original_inputs_id']=[tokenizer.convert_tokens_to_ids(sent) for sent in data['tokenized']]

In [184]:
def pad(sentlist,maxlen,PAD=0):
    padded_list = sentlist
    while(len(padded_list) < maxlen):
        padded_list.append(PAD)
    return padded_list

In [185]:
def mask(sentlist):
    attention_mask=[float(i>0) for i in sentlist]
    return attention_mask

In [186]:
#参数
MAX=17
data['inputs_id']=data['original_inputs_id'].apply(pad,maxlen=MAX)

In [187]:
data['attention_mask']=data['inputs_id'].apply(mask)

In [188]:
train_data=data.sample(frac=0.8,random_state=1)
test_data=data[~data.index.isin(train_data.index)]

In [189]:
train_inputs=torch.tensor(train_data['inputs_id'].tolist(),dtype=torch.int64)
train_labels=torch.tensor(train_data['fact'].tolist(),dtype=torch.int64)
train_masks=torch.tensor(train_data['attention_mask'].tolist(),dtype=torch.float)
test_inputs=torch.tensor(test_data['inputs_id'].tolist(),dtype=torch.int64)
test_labels=torch.tensor(test_data['fact'].tolist(),dtype=torch.int64)
test_masks=torch.tensor(test_data['attention_mask'].tolist(),dtype=torch.float)

In [190]:
batch_size = 32
train_data = torch.utils.data.TensorDataset(train_inputs, train_masks, train_labels)
train_iter = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_data = torch.utils.data.TensorDataset(test_inputs, test_masks, test_labels)
test_iter = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [191]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, batch):
        
        #inputs_ids = [batch size, sent len]
        input_ids,input_mask,_=batch
        
        with torch.no_grad():
            bert_out=self.bert(input_ids=input_ids)
            
            embedded = bert_out[0]
              
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [192]:
HIDDEN_DIM = 128
OUTPUT_DIM = 3
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
device = torch.device('cuda:0')
use_gpu = True
lr = 0.05
model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [193]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [194]:
optimizer = optim.Adam(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [195]:
def category_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.max(preds,1)[1]
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [196]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [197]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        batch = tuple(t.to(device) for t in batch)
        input_ids,input_mask,labels=batch
        
        optimizer.zero_grad()
        
        predictions = model(batch).squeeze(1)
        
        loss = criterion(predictions, labels)
        
        acc = category_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator),epoch_acc / len(iterator)

In [198]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            batch = tuple(t.to(device) for t in batch)
            input_ids,input_mask,labels=batch

            predictions = model(batch).squeeze(1)
            
            loss = criterion(predictions, labels)
            
            acc = category_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [199]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss,train_acc= train(model, train_iter, optimizer, criterion)
    valid_loss,valid_acc = evaluate(model, test_iter, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 1.423 | Train Acc: 51.97%
	 Val. Loss: 0.814 |  Val. Acc: 64.45%
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 1.145 | Train Acc: 52.38%
	 Val. Loss: 1.088 |  Val. Acc: 64.45%
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 1.173 | Train Acc: 51.44%
	 Val. Loss: 1.224 |  Val. Acc: 31.36%
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 1.235 | Train Acc: 52.02%
	 Val. Loss: 1.030 |  Val. Acc: 64.45%
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 1.181 | Train Acc: 52.89%
	 Val. Loss: 0.946 |  Val. Acc: 64.45%
Epoch: 06 | Epoch Time: 0m 2s
	Train Loss: 1.123 | Train Acc: 52.49%
	 Val. Loss: 0.824 |  Val. Acc: 64.45%
Epoch: 07 | Epoch Time: 0m 2s
	Train Loss: 1.180 | Train Acc: 54.59%
	 Val. Loss: 1.242 |  Val. Acc: 64.45%
Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 1.019 | Train Acc: 52.30%
	 Val. Loss: 1.000 |  Val. Acc: 64.45%
Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 1.044 | Train Acc: 55.94%
	 Val. Loss: 1.800 |  Val. Acc: 31.36%
Epoch: 10 | Epoch Time: 0m 2